[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tigarto/tarea-fundamentos/blob/main/entrega_udea_copy.ipynb)

In [ ]:
import pandas as pd 
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

from sklearn.impute import SimpleImputer

### LIMPIEZA DE DATOS

##### TAMAÑO DEL DF

In [ ]:
path='https://github.com/tigarto/tarea-fundamentos/raw/main/data_market.csv'
df=pd.read_csv(path,encoding='ISO-8859-1',delimiter=';',index_col='Row ID')

##### TAMAÑO DEL DF

In [ ]:
print(f'el tamaño del dataframe es de {df.shape[0]} filas y {df.shape[1]} columnas')

##### TIPO COLUMNAS

In [ ]:
columnas_numericas=df.select_dtypes(include=['number']).columns
columnas_categoricas=df.select_dtypes(include=['object']).columns
print(f'la cantidad de columnas numericas es {len(columnas_numericas)} y la de categoricas es {len(columnas_categoricas)}')

In [ ]:
df.dtypes

##### CAMBIAR TIPO STRING A FLOAT

In [ ]:
def remplazos(df):
    df['Profit'] = df['Profit'].str.replace('.','')
    df['Profit'] = df['Profit'].astype('Float64')
    df['Sales'] = df['Sales'].str.replace('.','')
    df['Sales'] = df['Sales'].astype('Float64')
    
    return df
    

In [ ]:
df=remplazos(df)

##### VALORES NULLOS 

In [ ]:
nullos=df.isnull().sum().reset_index()
nullos=nullos.rename(columns={'index':'columnas',0:'nullos_c'})
nullos=nullos[nullos.nullos_c>0]
nullos=nullos.set_index('columnas')
print(f'el numero de columnas con datos nullos es de {nullos.shape[0]}\
 para un total de {np.array(nullos.nullos_c).sum()} datos nullos en todo el data set')
nullos


##### FILTRAR VALORES NULLOS EN DF 

In [ ]:
filtered_df = df[df.isna().any(axis=1)]

##### % VALORES NULLOS 

In [ ]:
nullos['%']=round((nullos['nullos_c']/df.shape[0])*100,3)
nullos.loc['total'] = ['=',np.array(nullos['%']).sum()]
nullos

##### REMPLAZAR NULLOS POR PROMEDIO

In [ ]:
#df.replace(np.nan, 0,inplace=True)
# Reemplazar TSH por la media
mean_imputer = SimpleImputer(strategy='mean')
df.loc[:,['Sales','Quantity','Discount','Profit']] \
        = mean_imputer.fit_transform(df[['Sales','Quantity','Discount','Profit']])


In [ ]:
# SE VERIFICA QUE NO HAY VALORES NULLOS
df.isnull().values.any()

##### VALORES DUPLICADOS

In [ ]:
df.index.duplicated().sum()

##### SELECCION DE FEATURES Y AGREGAR FUNCIONES DE TIEMPO

In [ ]:
def tiempo(df):
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    df['año']=df['Ship Date'].dt.year
    df['mes']=df['Ship Date'].dt.month
    df['dia']=df['Ship Date'].dt.day
    return df

In [ ]:
# se borra pais por que solo tiene un solo country 
df__=df.loc[:,['Ship Date','Ship Mode','Customer Name','Segment','City','State','Category','Sub-Category','Quantity','Profit']]
df__=tiempo(df__)


##### VALORES UNICOS 

In [ ]:
def unicos(df):
    unicos={i:df[i].nunique()  for i in df.columns }
    return pd.DataFrame(unicos,index=['unicos'])\
            .T.sort_values(by='unicos',ascending=True)

In [ ]:
unicos_=unicos(df__)
unicos_

Al examinar los valores únicos, se puede detectar la diversidad y la distribución de los datos en cada variable, lo que ayuda a comprender mejor las relaciones y patrones presentes

##### MEDIDAS ESTADISTICAS  PROFIT

In [ ]:
promedio_profit=round(np.array(df__['Profit']).mean(),0)
max_profit=round(np.array(df__['Profit']).max(),0)
min_profit=round(np.array(df__['Profit']).min(),0)
desv_=round(np.array(df__['Profit']).std(),0)
Q1=np.percentile(np.array(df__['Profit']),25)
Q3=np.percentile(np.array(df__['Profit']),75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

##### RESUMEN ESTADISTICO

In [ ]:
resumen={
    'promedio_profit':promedio_profit,
    'max_profit':max_profit,
    'min_profit':min_profit,
    'desv_':desv_,
    'Q1':Q1,
    'Q3':Q3,
    'limite_inferior':limite_inferior,
    'limite_superior':limite_superior
}
resumen=pd.DataFrame(resumen,index=['resumen'])
resumen

##### AGRUPAR DATOS

##### conteo por segmento

In [ ]:
# segmento por cliente
segmento=df__.groupby(['Segment'])['Segment'].count()
segmento=segmento.reset_index(name='conteo')
segmento

##### conteo por ciudad y suma de profit

In [ ]:
ciudad=df__.groupby(['City'])['Profit'].sum()
ciudad=ciudad.reset_index(name='sum').sort_values(by='sum',ascending=False).head(21)
ciudad['valores'] = ciudad['sum'].apply(lambda x: '{:,.0f}'.format(x))
conteo=df__.groupby(['City'])['City'].count()
conteo=conteo.reset_index(name='conteo')
df_result=pd.merge(ciudad,conteo,on='City')
df_result=df_result.iloc[:,[0,2,3]]
df_result

##### profit estado

In [ ]:
estado=df__.groupby(['State'])['Profit'].sum()
estado=estado.reset_index(name='sum').sort_values(by='sum',ascending=False).head(21)
estado['valores'] = estado['sum'].apply(lambda x: '{:,.0f}'.format(x))
estado=estado.iloc[:,[0,2]]
estado

##### categorias mas frecuentes

In [ ]:
categoria=df__.groupby('Category')['Category'].count()
categoria=categoria.reset_index(name='conteo')
categoria_s=df__.groupby('Category')['Profit'].sum()
categoria_s=categoria_s.reset_index(name='suma')
categoria_s['valores'] = categoria_s['suma'].apply(lambda x: '{:,.0f}'.format(x))
categoria_s=pd.merge(categoria,categoria_s,on='Category')
categoria_s=categoria_s.iloc[:,[0,1,3]]
categoria_s

In [ ]:
sns.catplot(x = 'Category', y = 'conteo', data = categoria_s, kind = 'bar', hue = 'Category');
plt.title("Conteo por categoria")
plt.show()

##### Categorias discriminadas (Nota: Resultado raro)

In [ ]:
# Elementos por categoria y subcategoria
df_cat__ = df__.groupby(['Category','Sub-Category']).agg({'Sub-Category':'count', 'Profit':'sum'})
df_cat__.rename(columns = {'Sub-Category': 'conteo', 'Profit': 'valores'},inplace='True')
df_cat__

##### pedidos a lo largo del tiempo

In [ ]:
df.head(1)